# BIC- PSO ANN Optimisation

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import PySwarms as instructed by pyswarms webpage
import pyswarms as ps

%load_ext autoreload
%autoreload 2

## 1. Implement a multi-layer ANN architecture

### Data Preprocessing

#### Importing the dataset (Churn Modelling)

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [806 'France' 'Male' ... 1 1 142838.64]
 [757 'Germany' 'Male' ... 1 1 127059.04]
 [570 'France' 'Female' ... 0 1 116503.92]]


#### Encoding gender label

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

#### Encoding geography label

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Building the ANN

#### Initialising the ANN

In [5]:
#ANN architecture
n_inputs = 12
n_hidden = 20
n_classes = 2

num_samples = len(y)

In [6]:
#Add weight calculation
def logits_f(p):

    W1 = p[0:240].reshape((n_inputs,n_hidden))
    b1 = p[240:260].reshape((n_hidden,))
    W2 = p[260:300].reshape((n_hidden,n_classes))
    b2 = p[300:302].reshape((n_classes,))

# Forward propagation
    z1 = X.dot(W1) + b1 
    a1 = np.tanh(z1.astype(float))    
    logits = a1.dot(W2) + b2 
    return logits         

In [7]:
def f_prop(params):

    logits = logits_f(params)

# Softmax
    scores = np.exp(logits)
    probs = scores / np.sum(scores, axis=1, keepdims=True)

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss


In [8]:
def f(x):
    n_particles = x.shape[0]
    j = [f_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

## PSO Oprimisation with pyswarms

In [9]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9, 'k':2, 'p':2}

dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + 3*n_hidden + n_classes
optimizer = ps.single.LocalBestPSO(n_particles=50, dimensions=dimensions, options=options)

cost, pos = optimizer.optimize(f, iters=500)

2020-11-23 13:55:58,718 - pyswarms.single.local_best - INFO - Optimize for 500 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 2, 'p': 2}
pyswarms.single.local_best: 100%|██████████|500/500, best_cost=0.486
2020-11-23 13:57:48,274 - pyswarms.single.local_best - INFO - Optimization finished | best cost: 0.4860537795517748, best pos: [-0.38948862  0.62058464  0.54049379  0.70351343 -0.17398251  0.53269784
  1.00248901  0.07674998  0.52169167  0.00751177  0.30222026  0.69665302
  0.78148092  0.83307309  0.7220176  -0.08038299 -0.90947785 -0.85563994
 -0.34480684  0.068742    0.1108463  -0.27294563 -0.45072356  0.33447059
  0.19101363 -0.07640913  0.04073217  0.54610771 -0.0696881  -0.12047354
  0.10423682  0.19295416  0.27573802  0.81831682  0.35326656  0.105834
  0.12184845 -0.50624567  0.53831136 -0.08800036  0.15156822 -0.07253716
  0.18487573 -0.0443812   0.20308999  0.6708069   0.54149348  0.31425247
 -0.54251423  0.28938341 -0.35680026  0.53575897 -0.10948748  0.55132346
 -0.002625

In [12]:
##Prediction
def predict(pos):

    logits = logits_f(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [13]:
(predict(pos) == y).mean()

0.798